In [1]:
# 之前的介绍中，我们只依靠深度学习框架来完成训练的工作， 而忽略了操作参数的具体细节。 本节，我们将介绍以下内容：
# 1.访问参数，调试，诊断，优化
# 2.参数初始化
# 3.在不同组件中共享参数

In [2]:
import torch
from torch import nn

# 隐藏层的定义可以是：神经网络中不直接对应输入和输出的中间层
# 也就是说，这里的8的这一层就是单个的隐藏层
net = nn.Sequential(
    nn.Linear(4, 8), # layer 0
    nn.ReLU(), # layer 1
    nn.Linear(8, 1)) # layer 2
X = torch.rand(2, 4)
net(X)

tensor([[-0.1776],
        [-0.1717]], grad_fn=<AddmmBackward0>)

In [3]:
# 我们从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 
# 这就像模型是一个列表一样，每层的参数都在其属性中。 如下所示，我们可以检查第二个全连接层的参数
# state_dict 是 PyTorch 中所有 nn.Module 的内置方法, 用来查看参数， 保存模型， 加载模型
# 访问_modules, 这是一个Ordered_Dict

print(net[2].state_dict())
# 中括号是索引, 小括号是函数调用
print(net._modules["0"]) # 不建议用_modules查看参数，这是内部 API，未来 PyTorch 版本可能改变，不应该依赖pytorch的内部api

OrderedDict([('weight', tensor([[-0.1909, -0.3056, -0.1852, -0.2586,  0.2490, -0.2299,  0.2318,  0.1708]])), ('bias', tensor([0.0422]))])
Linear(in_features=4, out_features=8, bias=True)


In [4]:
# 注意，每个参数都表示为参数类的一个实例。 
# 要对参数执行任何操作，首先我们需要访问底层的数值。 有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
# 下面的代码从第二个全连接层（即第三个神经网络层）提取偏置， 提取后返回的是一个参数类实例，并进一步访问该参数的值。

print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0422], requires_grad=True)
tensor([0.0422])


In [5]:
# 参数是复合的对象，包含值、梯度和额外信息。 这就是我们需要显式参数值的原因。 
# 除了值之外，我们还可以访问每个参数的梯度。 
# 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

net[2].weight.grad == None

True

In [6]:
# 当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。 
# 当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂， 因为我们需要递归整个树来提取每个子块的参数。 
# 下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。

print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# named_parameters返回这个层的参数和名字
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'], net.state_dict()

(tensor([0.0422]),
 OrderedDict([('0.weight',
               tensor([[-0.3647, -0.2277, -0.1160,  0.0188],
                       [-0.2251, -0.1569, -0.0604, -0.3514],
                       [ 0.0318,  0.4238, -0.1661,  0.0714],
                       [-0.4852,  0.4074,  0.3562,  0.3262],
                       [ 0.2718,  0.3372, -0.0144,  0.2312],
                       [ 0.4786,  0.1589,  0.1778,  0.0387],
                       [ 0.1857, -0.4370, -0.4006,  0.1721],
                       [ 0.1909, -0.0787, -0.1145,  0.1738]])),
              ('0.bias',
               tensor([-0.4424,  0.0194,  0.1450, -0.2114, -0.4539,  0.0959, -0.4844,  0.1643])),
              ('2.weight',
               tensor([[-0.1909, -0.3056, -0.1852, -0.2586,  0.2490, -0.2299,  0.2318,  0.1708]])),
              ('2.bias', tensor([0.0422]))]))

In [8]:
# 让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。 我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。
# X = rand(2, 4)

def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1)) # 很明显，第一层net[0] = block2 , 第二层net[1] = Linear(4, 1)
rgnet(X)

tensor([[-0.2628],
        [-0.2628]], grad_fn=<AddmmBackward0>)

In [9]:
print(rgnet)
print("分割线")
# 如果是regnet[0][0].bias会报错，因为这样找的是一个Sequential，而Sequential没有bias属性，会报错
print(rgnet[0][0][0].weight)
print(rgnet[0][0][2].bias)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)
分割线
Parameter containing:
tensor([[ 0.0424,  0.3936, -0.3690, -0.4581],
        [ 0.1781,  0.0720,  0.3786, -0.2982],
        [ 0.0484,  0.2678,  0.278

In [10]:
# 知道了如何访问参数后，现在我们看看如何正确地初始化参数
# 深度学习框架提供默认随机初始化， 也允许我们创建自定义初始化方法， 满足我们通过其他规则实现初始化权重

In [11]:
# 内置初始化

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight, net[0].bias[0]

(Parameter containing:
 tensor([[ 0.0101, -0.0028,  0.0061,  0.0039],
         [ 0.0190,  0.0159, -0.0042, -0.0120],
         [-0.0034, -0.0102, -0.0216, -0.0031],
         [-0.0079, -0.0060, -0.0137, -0.0081],
         [-0.0091,  0.0074,  0.0100,  0.0185],
         [ 0.0076, -0.0064, -0.0034, -0.0096],
         [ 0.0130,  0.0137,  0.0157, -0.0043],
         [-0.0036,  0.0048,  0.0093, -0.0092]], requires_grad=True),
 tensor(0., grad_fn=<SelectBackward0>))

In [12]:
# 我们还可以将所有参数初始化为给定的常数，比如初始化为1。

def init_constant1(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant1)
net[0].weight[0]

tensor([1., 1., 1., 1.], grad_fn=<SelectBackward0>)

In [13]:
# 我们还可以对某些块应用不同的初始化方法。 
# 例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。

def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4505,  0.6658,  0.6187, -0.3078])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


##### 自定义初始化

In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        # * 在函数定义中，为打包成元组，比如def a(*args) ; 在函数调用中，表示解包成多个参数
        print("INIT", *[(name, param.shape) for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        with torch.no_grad():
            m.weight *= (m.weight.abs() >= 5)

net.apply(my_init)
net[0].weight[:2]

INIT ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
INIT ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))


tensor([[ 0.0000,  0.0000,  0.0000, -9.7328],
        [-6.7907, -7.9620, -0.0000, -8.7190]], grad_fn=<SliceBackward0>)

In [21]:
# 有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
net(X)
# 检查参数是否相同
print(net[2].weight[0] == net[4].weight[0])
net[2].weight.data[0, 0] = 100 # [0, 0]是对一个矩阵的直接索引；[0][0]是链式索引；两者结果一样的
# # 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight[0] == net[4].weight[0])

Parameter containing:
tensor([[ 0.1741, -0.1932,  0.0307, -0.1423,  0.0371, -0.3061, -0.0268, -0.0719],
        [ 0.0393,  0.0006,  0.2796, -0.2513, -0.3124,  0.2374, -0.0149,  0.0736],
        [ 0.1997, -0.1282, -0.1059,  0.2225,  0.2771,  0.3130,  0.1038, -0.2906],
        [-0.2793,  0.2147,  0.3526,  0.2102, -0.2894,  0.1419, -0.2685,  0.1945],
        [ 0.0266, -0.2051,  0.1307, -0.2139,  0.0129, -0.3394,  0.1733,  0.1424],
        [-0.1078,  0.0036, -0.0895, -0.3350,  0.3499,  0.1938, -0.2457, -0.2178],
        [-0.0190,  0.0541, -0.2866, -0.1933,  0.1341, -0.0320,  0.1166, -0.0659],
        [ 0.0364,  0.2059,  0.1534, -0.0258,  0.3113,  0.1280,  0.2600,  0.0470]],
       requires_grad=True)
tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
